🎯 KPI 3: Relación entre antigüedad como cliente y tasa de suscripción

In [8]:
# Requiere instalación una vez por entorno:
# !pip install matplotlib seaborn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [9]:
# 1 - Cargar el DataFrame
df= pd.read_excel("data_set_complete.xlsx")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39578 entries, 0 to 39577
Data columns (total 29 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   age                39578 non-null  int64         
 1   job                39578 non-null  object        
 2   marital            39578 non-null  object        
 3   education          39578 non-null  object        
 4   default            39578 non-null  int64         
 5   housing            39578 non-null  int64         
 6   loan               39578 non-null  int64         
 7   contact            39578 non-null  object        
 8   duration           39578 non-null  int64         
 9   campaign           39578 non-null  int64         
 10  pdays              39578 non-null  int64         
 11  previous           39578 non-null  int64         
 12  poutcome           39578 non-null  object        
 13  emp.var.rate       39578 non-null  float64       
 14  cons.p

In [10]:
# Asegurarse que las fechas están en formato datetime
df['Dt_Customer'] = pd.to_datetime(df['Dt_Customer'])
df['date'] = pd.to_datetime(df['date'])

# Crear columna de antigüedad en años (float, con decimales)
df['customer_years'] = (df['date'] - df['Dt_Customer']).dt.days / 365.25

# Corregimos los bins
bins = [0, 1, 3, 5, 10, np.inf]
labels = ['<1 año', '1-3 años', '3-5 años', '5-10 años', '10+ años']
df['antigüedad_rango'] = pd.cut(df['customer_years'], bins=bins, labels=labels, right=False)

# Verificación rápida
print("📊 Distribución de clientes por antigüedad:")
print(df['antigüedad_rango'].value_counts().sort_index())

# Vista previa
df[['Dt_Customer', 'date', 'customer_years', 'antigüedad_rango']].head()

📊 Distribución de clientes por antigüedad:
antigüedad_rango
<1 año        1489
1-3 años      9518
3-5 años     15595
5-10 años    12976
10+ años         0
Name: count, dtype: int64


,Dt_Customer,date,customer_years,antigüedad_rango
0,2014-07-17,2018-01-23,3.520876,3-5 años
1,2014-01-04,2018-12-02,4.908966,3-5 años
2,2014-12-01,2015-01-13,0.117728,<1 año
3,2012-01-25,2015-09-02,3.603012,3-5 años
4,2014-05-12,2018-04-30,3.967146,3-5 años


In [11]:
# Verificar antigüedad máxima y mínima
print("📅 Antigüedad mínima (años):", df['customer_years'].min())
print("📅 Antigüedad máxima (años):", df['customer_years'].max())

# Mostrar los clientes más antiguos
df.sort_values(by='customer_years', ascending=False).head()

📅 Antigüedad mínima (años): 0.0
📅 Antigüedad máxima (años): 7.997262149212868


,age,job,marital,education,default,housing,loan,contact,duration,campaign,...,longitude,uuid,Income,Kidhome,Teenhome,Dt_Customer,NumWebVisitsMonth,año,customer_years,antigüedad_rango
37809,39,unemployed,married,university degree,2,0,0,telephone,21,8,...,-85.297,f448652e-0753-4a68-b87d-64994fb93ae2,76106,2,0,2012-01-01,31,2012,7.997262,5-10 años
31153,30,admin.,single,high school,2,1,0,telephone,483,1,...,-86.474,c98a9ff3-3234-4c96-b639-a7e1c381465f,37096,1,1,2012-01-02,12,2012,7.972621,5-10 años
17935,38,admin.,married,high school,0,1,0,telephone,51,18,...,-79.191,7448ab04-2e04-47a3-b1dc-aaeddb7fb05c,23969,1,1,2012-01-12,19,2012,7.961670,5-10 años
32960,22,blue-collar,single,basic education,0,1,0,telephone,195,1,...,-97.414,d54541c8-bc3b-4234-901d-05b8c9cf4b6e,133129,0,1,2012-01-12,16,2012,7.958932,5-10 años
34897,31,entrepreneur,married,basic education,0,1,1,cellular,782,1,...,-122.802,e205dd4c-e2a4-4767-a0fb-2f01603a34cc,135027,2,2,2012-01-06,18,2012,7.958932,5-10 años


In [12]:
# Paso 1 - Agrupar por 'antigüedad_rango' y contar total de clientes y suscritos
agrupado = df.groupby('antigüedad_rango').agg(
    total_clientes=('y', 'count'),
    total_suscritos=('y', lambda x: (x == True).sum())
).reset_index()

/var/folders/xb/3yqyn6b15r7d87xw0gq3bhpm0000gn/T/ipykernel_20471/4237075225.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  agrupado = df.groupby('antigüedad_rango').agg(


In [13]:
# Paso 2 - Añadir columna con porcentaje de conversión
agrupado['porcentaje_suscripcion'] = (agrupado['total_suscritos'] / agrupado['total_clientes']) * 100

In [14]:
# Paso 3 - Mostrar resultados ordenados por antigüedad
display(agrupado.sort_values(by='antigüedad_rango'))

,antigüedad_rango,total_clientes,total_suscritos,porcentaje_suscripcion
0,<1 año,1489,279,18.737408
1,1-3 años,9518,1421,14.929607
2,3-5 años,15595,1658,10.631613
3,5-10 años,12976,928,7.151665
4,10+ años,0,0,NaN


. Insight del KPI 3

“La probabilidad de suscripción disminuye progresivamente cuanto más tiempo lleva un cliente en el banco. Los clientes nuevos (menos de 1 año) tienen una tasa de suscripción del 19.7 %, mientras que los de más de 5 años bajan al 7.15 %. No hay registros para clientes con más de 10 años.”